# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [151]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [152]:
weather_file = "../WeatherPy/weather_data.csv"
weather_file_df = pd.read_csv(weather_file)
weather_file_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,New Norfolk,-42.7826,147.0587,53.58,89,100,1.34,AU,1623718407
1,Rikitea,-23.1203,-134.9692,75.06,81,100,12.41,PF,1623718408
2,Port Hawkesbury,45.6169,-61.3485,53.73,95,90,5.99,CA,1623718409
3,Alyangula,-13.8483,136.4192,80.47,83,100,4.61,AU,1623718410
4,Puerto Ayora,-0.7393,-90.3518,73.54,86,100,3.00,EC,1623718410


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [153]:
gmaps.configure(api_key=g_key)

In [154]:
locations = weather_file_df[["Lat", "Lng"]].astype(float)
weight = weather_file_df["Humidity"].astype(float)

In [155]:
humidity_max = weather_file_df["Humidity"].max().astype(float)
humidity_max

100.0

In [156]:
fig = gmaps.figure(zoom_level=2, center=(-3.8,102))

heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=humidity_max, point_radius=2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [157]:
ideal_weather = weather_file_df.loc[((weather_file_df["Max Temp"] > 75) & (weather_file_df["Max Temp"] < 85)) &
                                  ((weather_file_df["Cloudiness"] < 50) & (weather_file_df["Cloudiness"] > 5)) &
                                   (weather_file_df["Wind Speed"] < 10) & (weather_file_df["Humidity"] < 75)]

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,Savelugu,9.6244,-0.8253,80.53,69,12,5.12,GH,1623718477
178,Shimoda,34.6667,138.9500,78.10,73,34,7.61,JP,1623718536
290,Kutum,14.2000,24.6667,79.39,18,40,5.97,SD,1623718618
338,Hamilton,39.1834,-84.5333,79.43,60,7,7.34,US,1623718654
353,Cairns,-16.9167,145.7667,83.10,74,40,1.14,AU,1623718529
456,Pangkalanbuun,-2.6833,111.6167,82.09,74,29,3.58,ID,1623718746
478,São Miguel do Araguaia,-13.2750,-50.1628,76.60,52,9,3.47,BR,1623718763
482,Springfield,37.2153,-93.2982,82.44,60,6,6.38,US,1623718672
526,Katsuura,35.1333,140.3000,77.04,71,21,6.96,JP,1623718795
541,Gizo Government Station,-8.1030,156.8419,83.77,73,32,5.66,SB,1623718811


In [158]:
all_indexes = weather_file_df[~((weather_file_df["Max Temp"] > 75) & (weather_file_df["Max Temp"] < 85) &
                                  ((weather_file_df["Cloudiness"] < 50) & (weather_file_df["Cloudiness"] > 5)) &
                                   (weather_file_df["Wind Speed"] < 10) & (weather_file_df["Humidity"] < 75))].index

weather_file_df.drop(all_indexes, inplace = True)

weather_file_df.dropna(how="any")

weather_file_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,Savelugu,9.6244,-0.8253,80.53,69,12,5.12,GH,1623718477
178,Shimoda,34.6667,138.9500,78.10,73,34,7.61,JP,1623718536
290,Kutum,14.2000,24.6667,79.39,18,40,5.97,SD,1623718618
338,Hamilton,39.1834,-84.5333,79.43,60,7,7.34,US,1623718654
353,Cairns,-16.9167,145.7667,83.10,74,40,1.14,AU,1623718529
456,Pangkalanbuun,-2.6833,111.6167,82.09,74,29,3.58,ID,1623718746
478,São Miguel do Araguaia,-13.2750,-50.1628,76.60,52,9,3.47,BR,1623718763
482,Springfield,37.2153,-93.2982,82.44,60,6,6.38,US,1623718672
526,Katsuura,35.1333,140.3000,77.04,71,21,6.96,JP,1623718795
541,Gizo Government Station,-8.1030,156.8419,83.77,73,32,5.66,SB,1623718811


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [159]:
hotel_df = weather_file_df
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
95,Savelugu,9.6244,-0.8253,80.53,69,12,5.12,GH,1623718477,,
178,Shimoda,34.6667,138.9500,78.10,73,34,7.61,JP,1623718536,,
290,Kutum,14.2000,24.6667,79.39,18,40,5.97,SD,1623718618,,
338,Hamilton,39.1834,-84.5333,79.43,60,7,7.34,US,1623718654,,
353,Cairns,-16.9167,145.7667,83.10,74,40,1.14,AU,1623718529,,
456,Pangkalanbuun,-2.6833,111.6167,82.09,74,29,3.58,ID,1623718746,,
478,São Miguel do Araguaia,-13.2750,-50.1628,76.60,52,9,3.47,BR,1623718763,,
482,Springfield,37.2153,-93.2982,82.44,60,6,6.38,US,1623718672,,
526,Katsuura,35.1333,140.3000,77.04,71,21,6.96,JP,1623718795,,
541,Gizo Government Station,-8.1030,156.8419,83.77,73,32,5.66,SB,1623718811,,


In [160]:
params = {"radius": 5000, "types": "lodging", "key": g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_detail = requests.get(base_url, params=params)
    
    hotel_detail = hotel_detail.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_detail["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_detail["results"][0]["vicinity"]
    
    except:
        print("Missing detail... skipping.")
    

In [161]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
95,Savelugu,9.6244,-0.8253,80.53,69,12,5.12,GH,1623718477,Yoo Naa's Palace,Savelugu
178,Shimoda,34.6667,138.9500,78.10,73,34,7.61,JP,1623718536,Shimoda Tokyu Hotel,"5-12-1, Shimoda"
290,Kutum,14.2000,24.6667,79.39,18,40,5.97,SD,1623718618,Dibbo areaمنطقة دبو,كتم،
338,Hamilton,39.1834,-84.5333,79.43,60,7,7.34,US,1623718654,Six Acres Bed & Breakfast,"5350 Hamilton Avenue, Cincinnati"
353,Cairns,-16.9167,145.7667,83.10,74,40,1.14,AU,1623718529,Mantra Trilogy Cairns,"101-105 Esplanade, Cairns City"
456,Pangkalanbuun,-2.6833,111.6167,82.09,74,29,3.58,ID,1623718746,Grand Kecubung Hotel,"Jalan Domba No.1, Mendawai"
478,São Miguel do Araguaia,-13.2750,-50.1628,76.60,52,9,3.47,BR,1623718763,Hotel Executivo Palace,"Avenida José Pereira do Nascimento, Quadra 46 ..."
482,Springfield,37.2153,-93.2982,82.44,60,6,6.38,US,1623718672,University Plaza Hotel & Convention Center,"333 South John Q Hammons Parkway, Springfield"
526,Katsuura,35.1333,140.3000,77.04,71,21,6.96,JP,1623718795,Katsuura Hotel Mikazuki,"820 Tona, Katsuura"
541,Gizo Government Station,-8.1030,156.8419,83.77,73,32,5.66,SB,1623718811,Imagination Island,"Imagination Island, Gizo"


In [162]:
hotel_name = hotel_df["Hotel Name"].tolist()

In [163]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [164]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(ideal_loc, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))